In [2]:
# -*- coding: utf-8 -*-

# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.
from jupyter_dash import JupyterDash
import dash
import pickle
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from pandas import Grouper
import datetime as dt
import plotly.io as pio
import json
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from functions_all import *

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [3]:
CO_county_json=json.load(open('data/CO_counties_geo.json', 'r'))

In [4]:
def make_menu_options(list_for_menu):
    """Making a list of dictionaries for menu choices out of a list of strings """
    options=[]
    for option in list_for_menu:
        options.append({'label': option, 'value': option})
    return options


def plot_crime(df, options=None):
    """Plots/replots crime with certain categories"""
    if options==None:
        rslt_df=df
    else:
        rslt_df = df[df['offense_category_name'].isin(options)]
    
    fig = px.line(rslt_df, x='timestamp', y='count', color='offense_category_name', 
                    color_discrete_map=color_discrete_map_,
                    labels={ 'timestamp': 'Date',
                           'count': 'Number of Offenses',
                           'offense_category_name': 'Offense Category'},
                    title='<b>Number of Offenses in Different Crime Categories</b>',
                    template='ggplot2'
                 )

    fig.update_layout(width=1100,
                      height=800)

    fig.update_layout(
        font_family="Serif",
        font_color="black",
        font_size=18,
        legend_font_size=13,
        title_font_family="Serif",
        title_font_size=22)

    fig.update_layout(
        title={
            'y':0.94,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        })

    fig.update_layout(
        xaxis=dict(
           rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         step='all',
                         stepmode='backward',
                         label='All'),
                    dict(count=5,
                         step='year',
                         stepmode= 'backward',
                         label='5Y'),
                    dict(count=1,
                         step='year',
                         stepmode= 'backward',
                         label='1Y')
                ])),
            rangeslider=dict(
                visible=True
            ),
        )
    )
    return fig

def plot_crime_against(df):
    """Function to plot crimes aginst bar chart"""
    
    
    fig = px.bar(df, x='crime_against', y='number_of_offenses', color='crime_against',
             animation_frame='year', animation_group='primary_county',
             hover_name="primary_county", hover_data=['number_of_offenses'], template='ggplot2',
             title='<b>Crime Against per Year and County</b>',
             labels={
                     'crime_against': 'Crime Against',
                     'number_of_offenses': 'Number of Offenses',
                     'primary_county': 'County an Offense Reported To',
                     'year': 'Year'})

    fig.update_layout(width=900,
                      height=600,
                      bargap=0.05)
    fig.update_layout(
        font_family="Serif",
        font_color="black",
        font_size=18,
        title_font_family="Serif",
        title_font_size=22)

    fig.update_layout(
        title={
            'y':0.94,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        })
    return fig

def plot_county_map(df):
    """Function to plot cholopleth of CO counties"""
    fig=px.choropleth_mapbox(data_frame=df, locations='primary_county', geojson=CO_county_json,
                                color='number_of_offenses', mapbox_style='carto-positron', zoom=6.13,
                                featureidkey='properties.name',
                                animation_frame='year', center={'lat': 39.0, 'lon': -105.5}, opacity=0.7,
                                color_continuous_scale=px.colors.sequential.Blues,
                                title='<b>Number of Offenses per County</b>',
                                labels={'primary_county': 'County',
                                        'number_of_offenses': 'Number of Offenses'},
                                template='ggplot2')
    fig.update_layout(
        font_family="Serif",
        font_color="black",
        font_size=18,
        legend_font_size=13,
        title_font_family="Serif",
        title_font_size=22)

    fig.update_layout(width=1100,
                      height=800)

    fig.update_layout(
        title={
            'y':0.94,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        })
    return fig

def create_dataframes(freq='M'):
    """Creating all dataframes used in the app and returning them, prodive frequency to group crime categories"""
    # Loading the main dataframe
    with open('data/pickled_dataframes/df_full_clean.pickle', 'rb') as f:
        df_full=pickle.load(f)

    # Creating a df to work with moving forward
    df_to_use=df_full.copy()
    df_to_use['year']=df_to_use['timestamp'].dt.year
    df_to_use['offense_id']=1
    df_to_use.drop(columns=['incident_id','agency_id'], inplace=True)

    #Creating a dataframe with 'crime agains' info to display
    df_grouped_crime_against = df_to_use.groupby(['primary_county', 'crime_against', 'year']).agg({'offense_id': ['count']})
    df_grouped_crime_against.columns = ['number_of_offenses']
    df_grouped_crime_against = df_grouped_crime_against.reset_index()
    df_grouped_crime_against=df_grouped_crime_against.sort_values(by=['year'])


    # Creating 'crime categories' dataframe
    colors_dark24=px.colors.qualitative.Dark24
    colors_dark24=colors_dark24[:-1]
    crime_categories=['Assault Offenses', 'Larceny/Theft Offenses', 
     'Drug/Narcotic Offenses', 'Fraud Offenses',
     'Destruction/Damage/Vandalism of Property', 
     'Burglary/Breaking & Entering', 'Sex Offenses', 
     'Arson', 'Motor Vehicle Theft', 'Kidnapping/Abduction',
     'Weapon Law Violations', 'Robbery',
     'Pornography/Obscene Material', 'Counterfeiting/Forgery', 
     'Bribery', 'Stolen Property Offenses', 'Prostitution Offenses',
     'Homicide Offenses', 'Extortion/Blackmail',
     'Embezzlement', 'Gambling Offenses',
     'Human Trafficking', 'Animal Cruelty']

    color_discrete_map_=dict(zip(crime_categories,colors_dark24))
    df_crime_categories = df_to_use.groupby(['offense_category_name', 
                              pd.Grouper(key='timestamp',
                                         freq=freq)])['offense_category_name'].agg(['count']).reset_index()
    df_crime_categories = df_crime_categories.sort_values(by=['timestamp', 'count'])


    # Creating a dataframe with crime per county info
    df_grouped_county = df_to_use.groupby(['primary_county', 'year']).agg({'offense_id': ['count']})
    df_grouped_county.columns = ['number_of_offenses']
    df_grouped_county = df_grouped_county.reset_index()
    df_grouped_county=df_grouped_county.sort_values(by=['year'])


    #Creating a dataframe  with crime per zip code indo
    df_grouped_zip = df_to_use.groupby(['primary_county', 'icpsr_zip','year']).agg({'offense_id': ['count']})
    df_grouped_zip.columns = ['number_of_offenses']
    df_grouped_zip = df_grouped_zip.reset_index()
    df_grouped_zip=df_grouped_zip.sort_values(by=['year'])
    
    return color_discrete_map_, df_to_use, df_grouped_crime_against, df_crime_categories, df_grouped_county, df_grouped_zip

In [5]:
color_discrete_map_,df_to_use,df_grouped_crime_against,df_crime_categories,df_grouped_county,df_grouped_zip=create_dataframes()

In [6]:
create_dataframes()[0]

{'Assault Offenses': '#2E91E5',
 'Larceny/Theft Offenses': '#E15F99',
 'Drug/Narcotic Offenses': '#1CA71C',
 'Fraud Offenses': '#FB0D0D',
 'Destruction/Damage/Vandalism of Property': '#DA16FF',
 'Burglary/Breaking & Entering': '#222A2A',
 'Sex Offenses': '#B68100',
 'Arson': '#750D86',
 'Motor Vehicle Theft': '#EB663B',
 'Kidnapping/Abduction': '#511CFB',
 'Weapon Law Violations': '#00A08B',
 'Robbery': '#FB00D1',
 'Pornography/Obscene Material': '#FC0080',
 'Counterfeiting/Forgery': '#B2828D',
 'Bribery': '#6C7C32',
 'Stolen Property Offenses': '#778AAE',
 'Prostitution Offenses': '#862A16',
 'Homicide Offenses': '#A777F1',
 'Extortion/Blackmail': '#620042',
 'Embezzlement': '#1616A7',
 'Gambling Offenses': '#DA60CA',
 'Human Trafficking': '#6C4516',
 'Animal Cruelty': '#0D2A63'}

In [66]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__) #external_stylesheets=external_stylesheets)


app.layout = html.Div(children=[
    
                                 html.H1(style={'color': 'black',
                                               'font-size': '40px',
                                                'font':'Ariel',
                                               'textAlign':'center'},
                                         children='Crime in Colorado'),
             
                                 html.Div(
                                          children=[
    
                                                    html.Div(style={"border":'none',
                                                                    'padding':'3px',
                                                                    'textAlign': 'center',
                                                                    'box-shadow': '0 8px 16px 0 rgba(0,0,0,0.2), 0 6px 20px 0 rgba(0,0,0,0.19)'},
                                                            id='menu',
                                                            children=[
                                                                html.H3(style={'color': 'black',
                                                                                'font-size': '20px',
                                                                                'font':'Ariel',
                                                                                'textAlign':'center'},
                                                                         children='Filter out by crime category and date'),
                                                                
                                                                
                                                                dcc.Dropdown(
                                                                     style={'color': 'black',
                                                                        'font-size': '20px',
                                                                        'font':'Ariel',
                                                                        'textAlign':'center',
                                                                        'margin-left': '80px',
                                                                        'width':'80%'},
                                                                     id='crime_categories_menu', 
                                                                     options=make_menu_options(list(df_crime_categories.offense_category_name.unique())),
                                                                     multi=True),
                                                                
                                                                html.H3(style={
                                                                            'font-size': '10px',
                                                                            'textAlign':'center'}),

                                                                 dcc.DatePickerRange(
                                                                        id='my-date-picker-range',
                                                                        start_date=dt.date(2009, 1, 1),
                                                                        end_date=dt.date(2020, 1, 1),
                                                                        min_date_allowed=dt.date(2009, 1, 1),
                                                                        max_date_allowed=dt.date(2020, 1, 1)),
                                                                
                                                                html.H3(style={'font-size':'10px',
                                                                              'textAlign':'center'}),
                                                                
                                                                html.Button('Submit',id='submit', n_clicks=0,
                                                                             style={'background-color': '#e7e7e7',
                                                                                    'border': 'black',
                                                                                    'color': 'black',
                                                                                    'padding': '20px 40px',
                                                                                    'text-align': 'center',
                                                                                    'button-align': 'center',
                                                                                    'text-decoration': 'none',
                                                                                    'display': 'inline-block',
                                                                                    'font-size': '20px',
                                                                                    'margin': '8px 6px',
                                                                                    'border-radius': '12px',
                                                                                    'box-shadow': '0 8px 16px 0 rgba(0,0,0,0.2), 0 6px 20px 0 rgba(0,0,0,0.19)'}
                                                                            )
                                                            ]
                                                            
                                                    ),
                                          html.Div(style={"border":'none',
                                                'padding':'3px',
                                                 'align': 'center'},
                                          children=[                                     
                                                dcc.Graph(
                                                          id='crime_categories',
                                                          figure=plot_crime(df_crime_categories))
                                                  ],
                                          
                                          ),                                    
                                                  
                                 dcc.Graph(
                                           id='crime_against',
                                           figure=plot_crime_against(df_grouped_crime_against)
                                          ),
       
                                 dcc.Graph(
                                           id='county_map',
                                           figure=plot_county_map(df_grouped_county)
                                          )
                                            ]
                            )
            ]
)

@app.callback(
    Output(component_id='crime_categories', component_property='figure'),
    [Input(component_id='submit', component_property='n_clicks')],
    [State(component_id='crime_categories_menu', component_property='value'),
     State(component_id='my-date-picker-range', component_property='start_date'),
     State(component_id='my-date-picker-range', component_property='end_date')])
def update_output_div(n_clicks, input_value, start_date, end_date):
    df=df_crime_categories.loc[((df_crime_categories.timestamp>start_date)&(df_crime_categories.timestamp<end_date))]
    return plot_crime(df, options=input_value)


#if __name__ == '__main__':

app.run_server(mode='inline', debug=True)